## NLP model creation and training

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.text import * 


The main thing here is [`RNNLearner`](/text.learner.html#RNNLearner). There are also some utility functions to help create and update text models.

## Quickly get a learner

In [ ]:
show_doc(language_model_learner)

<h4 id="language_model_learner" class="doc_header"><code>language_model_learner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L203" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#language_model_learner-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>language_model_learner</code>(**`data`**:[`DataBunch`](/basic_data.html#DataBunch), **`arch`**, **`config`**:`dict`=***`None`***, **`drop_mult`**:`float`=***`1.0`***, **`pretrained`**:`bool`=***`True`***, **`pretrained_fnames`**:`OptStrTuple`=***`None`***, **\*\*`learn_kwargs`**) → `LanguageLearner`

<div class="collapse" id="language_model_learner-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#language_model_learner-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>language_model_learner</code>:</p><ul><li><code>pytest -sv tests/test_text_train.py::test_qrnn_works_if_split_fn_provided</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_text_train.py#L73" class="source_link" style="float:right">[source]</a></li><li><code>pytest -sv tests/test_text_train.py::test_qrnn_works_with_no_split</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_text_train.py#L61" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Create a [`Learner`](/basic_train.html#Learner) with a language model from `data` and `arch`.  

The model used is given by `arch` and `config`. It can be:

- an [<code>AWD_LSTM</code>](/text.models.html#AWD_LSTM)([Merity et al.](https://arxiv.org/abs/1708.02182))
- a [<code>Transformer</code>](/text.models.html#Transformer) decoder ([Vaswani et al.](https://arxiv.org/abs/1706.03762))
- a [<code>TransformerXL</code>](/text.models.html#TransformerXL) ([Dai et al.](https://arxiv.org/abs/1901.02860))

They each have a default config for language modelling that is in <code>{lower_case_class_name}\_lm\_config</code> if you want to change the default parameter. At this stage, only the AWD LSTM and Tranformer support `pretrained=True` but we hope to add more pretrained models soon. `drop_mult` is applied to all the dropouts weights of the `config`, `learn_kwargs` are passed to the [`Learner`](/basic_train.html#Learner) initialization.

If your [`data`](/text.data.html#text.data) is backward, the pretrained model downloaded will also be a backward one (only available for [`AWD_LSTM`](/text.models.awd_lstm.html#AWD_LSTM)).

In [ ]:
jekyll_note("Using QRNN (change the flag in the config of the AWD LSTM) requires to have cuda installed (same version as pytorch is using).")

<div markdown="span" class="alert alert-info" role="alert"><i class="fa fa-info-circle"></i> <b>Note: </b>Using QRNN (change the flag in the config of the AWD LSTM) requires to have cuda installed (same version as pytorch is using).</div>

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
data = TextLMDataBunch.from_csv(path, 'texts.csv')
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5)

In [ ]:
show_doc(text_classifier_learner)

<h4 id="text_classifier_learner" class="doc_header"><code>text_classifier_learner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L289" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#text_classifier_learner-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>text_classifier_learner</code>(**`data`**:[`DataBunch`](/basic_data.html#DataBunch), **`arch`**:`Callable`, **`bptt`**:`int`=***`70`***, **`max_len`**:`int`=***`1400`***, **`config`**:`dict`=***`None`***, **`pretrained`**:`bool`=***`True`***, **`drop_mult`**:`float`=***`1.0`***, **`lin_ftrs`**:`Collection`\[`int`\]=***`None`***, **`ps`**:`Collection`\[`float`\]=***`None`***, **\*\*`learn_kwargs`**) → `TextClassifierLearner`

<div class="collapse" id="text_classifier_learner-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#text_classifier_learner-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>text_classifier_learner</code>:</p><ul><li><code>pytest -sv tests/test_text_train.py::test_classifier</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_text_train.py#L100" class="source_link" style="float:right">[source]</a></li><li><code>pytest -sv tests/test_text_train.py::test_order_preds</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_text_train.py#L139" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Create a [`Learner`](/basic_train.html#Learner) with a text classifier from `data` and `arch`.  

Here again, the backbone of the model is determined by `arch` and `config`. The input texts are fed into that model by bunch of `bptt` and only the last `max_len` activations are considered. This gives us the backbone of our model. The head then consists of:
- a layer that concatenates the final outputs of the RNN with the maximum and average of all the intermediate outputs (on the sequence length dimension),
- blocks of ([`nn.BatchNorm1d`](https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm1d), [`nn.Dropout`](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout), [`nn.Linear`](https://pytorch.org/docs/stable/nn.html#torch.nn.Linear), [`nn.ReLU`](https://pytorch.org/docs/stable/nn.html#torch.nn.ReLU)) layers.

The blocks are defined by the `lin_ftrs` and `drops` arguments. Specifically, the first block will have a number of inputs inferred from the backbone arch and the last one will have a number of outputs equal to data.c (which contains the number of classes of the data) and the intermediate blocks have a number of inputs/outputs determined by `lin_ftrs` (of course a block has a number of inputs equal to the number of outputs of the previous block). The dropouts all have the same value ps if you pass a float, or the corresponding values if you pass a list. Default is to have an intermediate hidden size of 50 (which makes two blocks model_activation -> 50 -> n_classes) with a dropout of 0.1.

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
data = TextClasDataBunch.from_csv(path, 'texts.csv')
learn = text_classifier_learner(data, AWD_LSTM, drop_mult=0.5)

In [ ]:
show_doc(RNNLearner)

<h2 id="RNNLearner" class="doc_header"><code>class</code> <code>RNNLearner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L45" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#RNNLearner-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>RNNLearner</code>(**`data`**:[`DataBunch`](/basic_data.html#DataBunch), **`model`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`split_func`**:`OptSplitFunc`=***`None`***, **`clip`**:`float`=***`None`***, **`alpha`**:`float`=***`2.0`***, **`beta`**:`float`=***`1.0`***, **`metrics`**=***`None`***, **\*\*`learn_kwargs`**) :: [`Learner`](/basic_train.html#Learner)

<div class="collapse" id="RNNLearner-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#RNNLearner-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>RNNLearner</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Basic class for a [`Learner`](/basic_train.html#Learner) in NLP.  

Handles the whole creation from <code>data</code> and a `model` with a text data using a certain `bptt`. The `split_func` is used to properly split the model in different groups for gradual unfreezing and differential learning rates. Gradient clipping of `clip` is optionally applied. `alpha` and `beta` are all passed to create an instance of [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer). Can be used for a language model or an RNN classifier. It also handles the conversion of weights from a pretrained model as well as saving or loading the encoder.

In [ ]:
show_doc(RNNLearner.get_preds)

<h4 id="RNNLearner.get_preds" class="doc_header"><code>get_preds</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L85" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#RNNLearner-get_preds-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>get_preds</code>(**`ds_type`**:[`DatasetType`](/basic_data.html#DatasetType)=***`<DatasetType.Valid: 2>`***, **`activ`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)=***`None`***, **`with_loss`**:`bool`=***`False`***, **`n_batch`**:`Optional`\[`int`\]=***`None`***, **`pbar`**:`Union`\[`MasterBar`, `ProgressBar`, `NoneType`\]=***`None`***, **`ordered`**:`bool`=***`True`***) → `List`\[`Tensor`\]

<div class="collapse" id="RNNLearner-get_preds-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#RNNLearner-get_preds-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>get_preds</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Return predictions and targets on the valid, train, or test set, depending on `ds_type`.  

If `ordered=True`, returns the predictions in the order of the dataset, otherwise they will be ordered by the sampler (from the longest text to the shortest). The other arguments are passed to [`Learner.get_preds`](/basic_train.html#Learner.get_preds).

In [ ]:
show_doc(TextClassificationInterpretation,title_level=3)

<h3 id="TextClassificationInterpretation" class="doc_header"><code>class</code> <code>TextClassificationInterpretation</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/interpret.py#L35" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#TextClassificationInterpretation-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>TextClassificationInterpretation</code>(**`learn`**:[`Learner`](/basic_train.html#Learner), **`preds`**:`Tensor`, **`y_true`**:`Tensor`, **`losses`**:`Tensor`, **`ds_type`**:[`DatasetType`](/basic_data.html#DatasetType)=***`<DatasetType.Valid: 2>`***) :: [`ClassificationInterpretation`](/train.html#ClassificationInterpretation)

<div class="collapse" id="TextClassificationInterpretation-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#TextClassificationInterpretation-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>TextClassificationInterpretation</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Provides an interpretation of classification based on input sensitivity. This was designed for AWD-LSTM only for the moment, because Transformer already has its own attentional model. 

The darker the word-shading in the below example, the more it contributes to the classification.  Results here are without any fitting.  After fitting to acceptable accuracy, this class can show you what is being used to produce the classification of a particular case.

In [ ]:
import matplotlib.cm as cm

txt_ci = TextClassificationInterpretation.from_learner(learn)
test_text = "Zombiegeddon was perhaps the GREATEST movie i have ever seen!"
txt_ci.show_intrinsic_attention(test_text,cmap=cm.Purples)

You can also view the raw attention values with `.intrinsic_attention(text)`

In [ ]:
txt_ci.intrinsic_attention(test_text)[1]

tensor([0.6078, 0.4961, 0.4707, 0.4946, 0.5228, 0.5393, 0.5656, 0.6153, 0.6893,
        0.8047, 0.9329, 1.0000, 0.9080, 0.5786], device='cuda:0')

Create a tabulation showing the first `k` texts in top_losses along with their prediction, actual,loss, and probability of actual class. `max_len` is the maximum number of tokens displayed. If `max_len=None`, it will display all tokens.

In [ ]:
txt_ci.show_top_losses(5)

Text,Prediction,Actual,Loss,Probability
"xxbos i have to agree with what many of the other reviewers concluded . a subject which could have been thought - provoking and shed light on a reversed double - standard , failed miserably . \n \n xxmaj rape being a crime of violence and forced abusive control , the scenes here were for the most part pathetic . xxmaj it would have been a better idea to",pos,neg,8.25,0.00
"xxbos xxmaj betty xxmaj sizemore ( xxmaj renee xxmaj zellweger ) lives her life through soap xxmaj opera "" a xxmaj reason to xxmaj love "" as a way to escape her slob husband and dull life . xxmaj after a shocking incident involving two hit men ( xxmaj morgan xxmaj freeman and xxmaj chris xxmaj rock ) , xxmaj betty goes into shock and travels to xxup la ,",pos,pos,7.71,1.00
"xxbos xxmaj when people harp on about how "" they do n't make 'em like they used to "" then just point them towards this fantastically entertaining , and quaint - looking , comedy horror from writer - director xxmaj glenn mcquaid . \n \n xxmaj it 's a tale of graverobbers ( played by xxmaj dominic xxmaj monaghan and xxmaj larry xxmaj fessenden ) who end up digging",pos,pos,7.47,1.00
"xxbos i have to agree with all the previous xxunk -- this is simply the best of all frothy comedies , with xxmaj bardot as sexy as xxmaj marilyn xxmaj monroe ever was , and definitely with a prettier face ( maybe there 's less mystique , but look how xxmaj marilyn paid for that . ) i do n't think i 've ever seen such a succulent - looking",pos,pos,6.55,1.00
"xxbos i will freely admit that i have n't seen the original movie , but i 've read the play , so i 've some background with the "" original . "" xxmaj if you shuck off the fact that this is a remake of an old classic , this movie is smart , witty , fresh , and hilarious . xxmaj yes , the casting decisions may seem strange",pos,pos,6.38,1.00


### Loading and saving

In [ ]:
show_doc(RNNLearner.load_encoder)

<h4 id="RNNLearner.load_encoder" class="doc_header"><code>load_encoder</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L65" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#RNNLearner-load_encoder-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>load_encoder</code>(**`name`**:`str`, **`device`**:[`device`](https://pytorch.org/docs/stable/tensor_attributes.html#torch-device)=***`None`***)

<div class="collapse" id="RNNLearner-load_encoder-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#RNNLearner-load_encoder-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>load_encoder</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Load the encoder `name` from the model directory.  

In [ ]:
show_doc(RNNLearner.save_encoder)

<h4 id="RNNLearner.save_encoder" class="doc_header"><code>save_encoder</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L57" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#RNNLearner-save_encoder-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>save_encoder</code>(**`name`**:`str`)

<div class="collapse" id="RNNLearner-save_encoder-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#RNNLearner-save_encoder-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>save_encoder</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Save the encoder to `name` inside the model directory.  

In [ ]:
show_doc(RNNLearner.load_pretrained)

<h4 id="RNNLearner.load_pretrained" class="doc_header"><code>load_pretrained</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L75" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#RNNLearner-load_pretrained-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>load_pretrained</code>(**`wgts_fname`**:`str`, **`itos_fname`**:`str`, **`strict`**:`bool`=***`True`***)

<div class="collapse" id="RNNLearner-load_pretrained-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#RNNLearner-load_pretrained-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>load_pretrained</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Load a pretrained model and adapts it to the data vocabulary.  

Opens the weights in the `wgts_fname` of `self.model_dir` and the dictionary in `itos_fname` then adapts the pretrained weights to the vocabulary of the <code>data</code>. The two files should be in the models directory of the `learner.path`.

## Utility functions

In [ ]:
show_doc(convert_weights)

<h4 id="convert_weights" class="doc_header"><code>convert_weights</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L28" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#convert_weights-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>convert_weights</code>(**`wgts`**:`Weights`, **`stoi_wgts`**:`Dict`\[`str`, `int`\], **`itos_new`**:`StrList`) → `Weights`

<div class="collapse" id="convert_weights-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#convert_weights-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>convert_weights</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Convert the model `wgts` to go with a new vocabulary.  

Uses the dictionary `stoi_wgts` (mapping of word to id) of the weights to map them to a new dictionary `itos_new` (mapping id to word).

## Get predictions

In [ ]:
show_doc(LanguageLearner, title_level=3)

<h3 id="LanguageLearner" class="doc_header"><code>class</code> <code>LanguageLearner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L117" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#LanguageLearner-pytest" style="float:right; padding-right:10px">[test]</a></h3>

> <code>LanguageLearner</code>(**`data`**:[`DataBunch`](/basic_data.html#DataBunch), **`model`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`split_func`**:`OptSplitFunc`=***`None`***, **`clip`**:`float`=***`None`***, **`alpha`**:`float`=***`2.0`***, **`beta`**:`float`=***`1.0`***, **`metrics`**=***`None`***, **\*\*`learn_kwargs`**) :: [`RNNLearner`](/text.learner.html#RNNLearner)

<div class="collapse" id="LanguageLearner-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#LanguageLearner-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>LanguageLearner</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Subclass of RNNLearner for predictions.  

In [ ]:
show_doc(LanguageLearner.predict)

<h4 id="LanguageLearner.predict" class="doc_header"><code>predict</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L120" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#LanguageLearner-predict-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>predict</code>(**`text`**:`str`, **`n_words`**:`int`=***`1`***, **`no_unk`**:`bool`=***`True`***, **`temperature`**:`float`=***`1.0`***, **`min_p`**:`float`=***`None`***, **`sep`**:`str`=***`' '`***, **`decoder`**=***`'decode_spec_tokens'`***)

<div class="collapse" id="LanguageLearner-predict-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#LanguageLearner-predict-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>predict</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Return `text` and the `n_words` that come after  

If `no_unk=True` the unknown token is never picked. Words are taken randomly with the distribution of probabilities returned by the model. If `min_p` is not `None`, that value is the minimum probability to be considered in the pool of words. Lowering `temperature` will make the texts less randomized. 

In [ ]:
show_doc(LanguageLearner.beam_search)

<h4 id="LanguageLearner.beam_search" class="doc_header"><code>beam_search</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L140" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#LanguageLearner-beam_search-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>beam_search</code>(**`text`**:`str`, **`n_words`**:`int`, **`no_unk`**:`bool`=***`True`***, **`top_k`**:`int`=***`10`***, **`beam_sz`**:`int`=***`1000`***, **`temperature`**:`float`=***`1.0`***, **`sep`**:`str`=***`' '`***, **`decoder`**=***`'decode_spec_tokens'`***)

<div class="collapse" id="LanguageLearner-beam_search-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#LanguageLearner-beam_search-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>beam_search</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Return the `n_words` that come after `text` using beam search.  

## Basic functions to get a model

In [ ]:
show_doc(get_language_model)

<h4 id="get_language_model" class="doc_header"><code>get_language_model</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L189" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#get_language_model-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>get_language_model</code>(**`arch`**:`Callable`, **`vocab_sz`**:`int`, **`config`**:`dict`=***`None`***, **`drop_mult`**:`float`=***`1.0`***)

<div class="collapse" id="get_language_model-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#get_language_model-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>get_language_model</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create a language model from `arch` and its `config`, maybe `pretrained`.  

In [ ]:
show_doc(get_text_classifier)

<h4 id="get_text_classifier" class="doc_header"><code>get_text_classifier</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L272" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#get_text_classifier-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>get_text_classifier</code>(**`arch`**:`Callable`, **`vocab_sz`**:`int`, **`n_class`**:`int`, **`bptt`**:`int`=***`70`***, **`max_len`**:`int`=***`1400`***, **`config`**:`dict`=***`None`***, **`drop_mult`**:`float`=***`1.0`***, **`lin_ftrs`**:`Collection`\[`int`\]=***`None`***, **`ps`**:`Collection`\[`float`\]=***`None`***, **`pad_idx`**:`int`=***`1`***) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

<div class="collapse" id="get_text_classifier-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#get_text_classifier-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>get_text_classifier</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create a text classifier from `arch` and its `config`, maybe `pretrained`.  

This model uses an encoder taken from the `arch` on `config`. This encoder is fed the sequence by successive bits of size `bptt` and we only keep the last `max_seq` outputs for the pooling layers.

The decoder use a concatenation of the last outputs, a `MaxPooling` of all the outputs and an `AveragePooling` of all the outputs. It then uses a list of `BatchNorm`, `Dropout`, `Linear`, `ReLU` blocks (with no `ReLU` in the last one), using a first layer size of `3*emb_sz` then following the numbers in `n_layers`. The dropouts probabilities are read in `drops`.

Note that the model returns a list of three things, the actual output being the first, the two others being the intermediate hidden states before and after dropout (used by the [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer)). Most loss functions expect one output, so you should use a Callback to remove the other two if you're not using [`RNNTrainer`](/callbacks.rnn.html#RNNTrainer).

## Undocumented Methods - Methods moved below this line will intentionally be hidden

## New Methods - Please document or move to the undocumented section

In [ ]:
show_doc(MultiBatchEncoder.forward)

<h4 id="MultiBatchEncoder.forward" class="doc_header"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L260" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MultiBatchEncoder-forward-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>forward</code>(**`input`**:`LongTensor`) → `Tuple`\[`List`\[`Tensor`\], `List`\[`Tensor`\], `Tensor`\]

<div class="collapse" id="MultiBatchEncoder-forward-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MultiBatchEncoder-forward-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>forward</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:[`Module`](/torch_core.html#Module) instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(LanguageLearner.show_results)

<h4 id="LanguageLearner.show_results" class="doc_header"><code>show_results</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L167" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#LanguageLearner-show_results-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>show_results</code>(**`ds_type`**=***`<DatasetType.Valid: 2>`***, **`rows`**:`int`=***`5`***, **`max_len`**:`int`=***`20`***)

<div class="collapse" id="LanguageLearner-show_results-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#LanguageLearner-show_results-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>show_results</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Show `rows` result of predictions on `ds_type` dataset.  

In [ ]:
show_doc(MultiBatchEncoder.concat)

<h4 id="MultiBatchEncoder.concat" class="doc_header"><code>concat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L253" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MultiBatchEncoder-concat-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>concat</code>(**`arrs`**:`Sequence`\[`Sequence`\[`Tensor`\]\]) → `List`\[`Tensor`\]

<div class="collapse" id="MultiBatchEncoder-concat-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MultiBatchEncoder-concat-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>concat</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Concatenate the `arrs` along the batch dimension.  

In [ ]:
show_doc(MultiBatchEncoder)

<h2 id="MultiBatchEncoder" class="doc_header"><code>class</code> <code>MultiBatchEncoder</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L248" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MultiBatchEncoder-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>MultiBatchEncoder</code>(**`bptt`**:`int`, **`max_len`**:`int`, **`module`**:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), **`pad_idx`**:`int`=***`1`***) :: [`PrePostInitMeta`](/core.html#PrePostInitMeta) :: [`Module`](/torch_core.html#Module)

<div class="collapse" id="MultiBatchEncoder-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MultiBatchEncoder-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>MultiBatchEncoder</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create an encoder over `module` that can process a full sentence.  

In [ ]:
show_doc(decode_spec_tokens)

<h4 id="decode_spec_tokens" class="doc_header"><code>decode_spec_tokens</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L98" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#decode_spec_tokens-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>decode_spec_tokens</code>(**`tokens`**)

<div class="collapse" id="decode_spec_tokens-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#decode_spec_tokens-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>decode_spec_tokens</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

In [ ]:
show_doc(MultiBatchEncoder.reset)

<h4 id="MultiBatchEncoder.reset" class="doc_header"><code>reset</code><a href="https://github.com/fastai/fastai/blob/master/fastai/text/learner.py#L257" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#MultiBatchEncoder-reset-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>reset</code>()

<div class="collapse" id="MultiBatchEncoder-reset-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#MultiBatchEncoder-reset-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>reset</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>